In [1]:
from langdetect import detect
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np
import os

In [2]:
df = pd.read_csv("../data/qald-3-test.csv")

In [3]:
df= df.dropna(subset=['questions'])

In [4]:
df["lang"].unique()

array(['en', 'de', 'es', 'it', 'fr', 'nl'], dtype=object)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(df['questions'])

from sklearn.feature_extraction.text import TfidfTransformer
 
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
X = tfidf_transformer.fit_transform(X_train)

# get feature names
feature_names=vectorizer.get_feature_names()

#Function for sorting tf_idf in descending order
from scipy.sparse import coo_matrix
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results
def ExtractKeyWords(text):
    #generate tf-idf for the given text
    tf_idf_vector=tfidf_transformer.transform(text)
    
    
    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())
    #extract only the top n; n here is 3
    keywords=extract_topn_from_vector(feature_names,sorted_items,3)
 
    # now print the results
    sentence = ""
    for k in keywords:
        sentence+=(" "+k)
    return sentence

new_set = []
for text in X:
    new_set.append(ExtractKeyWords(text))
    
df["questions"]=new_set

In [6]:
with open('opennlp_input.txt', 'w') as f:
    for q in df["questions"]:
        f.write(q+"\n\n")

In [7]:
# Run openNLP Language Detector
import datetime
a = datetime.datetime.now()
os.system("/home/oshara/GSoC/DBPedia/openNLP/apache-opennlp-1.9.2/bin/opennlp LanguageDetector /home/oshara/GSoC/DBPedia/openNLP/langdetect-183.bin </home/oshara/GSoC/DBPedia/QLang/evaluation/opennlp_input.txt> /home/oshara/GSoC/DBPedia/QLang/evaluation/opennlp_output.txt")
b = datetime.datetime.now()
c = b-a
c/df.shape[0]

datetime.timedelta(microseconds=1638)

In [8]:
result=[]
count = 0
def extractLan(f):
    global result
    global count
    while count < df.shape[0]:
        line = f.readline()
        if line == "\n":
            continue
        result.append(line[:3])
        count += 1
        
with open('opennlp_output.txt', 'r') as f:
    extractLan(f)

In [9]:
# English, Spanish, German, Italian, French, Dutch, Romanian
# {'en': 0, 'de': 1, 'es': 2, 'it': 3, 'fr': 4, 'nl': 5, 'ro': 6}
label = {'eng': 0, "spa": 2, "deu": 1, "ita": 3, "fra": 4, "nld": 5, "ron": 6, "other": 7}
# label = {'eng': 0,"fas":1 ,"spa": 3, "deu": 2, "ita": 4, "fra": 5, "nld": 6, "ron": 7, "other": 8}
# label = {'eng': 0, "deu": 1, "ita": 2, "fra": 3,"other": 4}

In [10]:
for i in range(len(result)):
    key = result[i]
    if key in label:
        result[i] = label[key]
    else:
        result[i] = 7

In [11]:
possible_labels = df["lang"].unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

df['lang'] = df.lang.replace(label_dict)

In [12]:
y_true = df['lang']

In [13]:
q7 = pd.read_csv("../data/qald-7-train.csv")

In [14]:
# languages
# q3 ['en', 'de', 'es', 'it', 'fr', 'nl']
# q4 ['en', 'de', 'es', 'it', 'fr', 'nl', 'ro']
# q5 ['en', 'de', 'es', 'it', 'fr', 'nl', 'ro']
# q6 ['en', 'fa', 'de', 'es', 'it', 'fr', 'nl', 'ro']
# q7 ['en', 'de', 'it', 'fr']
# q8 ['en']

In [15]:
# total times spent and accuracy and rows
# q3 0.61
# q4 0.69
# q5 0.65
# q6 0.5675
# q7 0.62
# q8 0.46

In [16]:
df["lang"].unique()

array([0, 1, 2, 3, 4, 5])

In [17]:
accuracy_score(y_true, result)

0.6144781144781145

##### 